# Level advancements E-Shuleni

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import plotly.graph_objects as go

**Process**

- Load every friday the csv of all 4 streams and save them in folder track levels
- Code reads the whole folder, segment the files per stream, concatenate all files of same stream together, groupby name and aggregate the (sync date, Level) for all kids.
- Plot each stream in a plot with time on x axis and level on y axis with levels order being: ['Introductory, 1, 2, ..., 21]

### Load data

In [2]:
DIR_LEVELS = "./../Pilot 2025/Data Enuma LMS/track levels/"
DIR_PLOTS_DASHBOARD = "./../E-Shuleni-Dashboard/plots/"
DIR = "./../Pilot 2025/Data Enuma LMS/"

#all_kids_2025 = pd.read_excel("./../Pilot 2025/list_students_2025.xlsx")
#all_kids_2025 = pd.read_excel("./../Pilot 2025/E-Shuleni Student Roster - 2025.xlsx")
#print(f"Numbe of students in Pilot 2025: {all_kids_2025.shape[0]}")

#if all_kids_2025.shape[0] != all_kids_2025["STUDENTS NAME"].nunique():
#    print("Some students are duplicated")
#all_kids_2025.head(2)

# kids left the program
oop_kids = pd.read_excel('./../Pilot 2025/Data Enuma LMS/students_out_of_program_2025.xlsx')
print(f"Number of students who left the program: {oop_kids.shape[0]}\n{oop_kids['Name'].values.tolist()}")

# Absent students
absenteeism_table = pd.read_excel(DIR + f"/absenteeism table.xlsx")
# reformat columns with date to YYYY-MM-DD format
new_cols = ["Student", "Class"] + [pd.to_datetime(col).strftime('%Y-%m-%d') for col in absenteeism_table.columns[2:]]
absenteeism_table.columns = new_cols
# remove kids that left the program from absenteeism table
absenteeism_table = absenteeism_table[~absenteeism_table["Student"].isin(oop_kids["Name"].values.tolist())].reset_index(drop=True)#.set_index("Student")
print(f"Number of kids in program (from absenteeism table): {len(absenteeism_table)}")
absenteeism_table.head(2)

Number of students who left the program: 20
['ADRIAN JULIUS', 'BRAVIAN NANDA', 'BRAYTNESS MTABIRO', 'JUNIOUR MBISE', 'MELKIZEDECK AKYOO', 'PRAYGOD AKYOO', 'YUNISI AKYOO', 'Naomi Ibrahim Mamusi', 'RICKSON KITOMARI', 'MAURINE NOEL KYUNGAI', 'SHEDRACK MTISYA', 'Johnson Mollel', 'MARY KITOMARI', 'MISHELI PETER KYUNGAI', 'ARAFAT JONAS', 'SHEDRACK KITOMA RI', 'SHEDRACK KITOMARI', 'MAXMILLIAN VENANCE  REVOCATUS', 'BRAYSON CHARLES KITOMARI', 'LISA EZEKIEL PALLANGYO']
Number of kids in program (from absenteeism table): 158


,Student,Class,2025-02-07,2025-02-14,2025-02-21,2025-02-28,2025-03-07,2025-03-14,2025-03-21
0,Beautiful Godman Akyoo,4A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRAYAN JOSHUA KIMUTHO,4A,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# process absenteeism table to have dict with dates and list of absent students
absenteeism_dict = {}
for col in absenteeism_table.columns[2:]:
    absent_students = absenteeism_table[absenteeism_table[col] == "X"]["Student"].tolist()
    absenteeism_dict[col] = absent_students
for key, value in absenteeism_dict.items():
    print(f"{key}: {len(value)} students absent: {value}")

#absenteeism_table.info()

2025-02-07: 0 students absent: []
2025-02-14: 0 students absent: []
2025-02-21: 2 students absent: ['HILARY NDEWIRWA AKYOO', 'ELISHA GERSON KITOMARI']
2025-02-28: 7 students absent: ['ANJELINA ISSA DUME', 'DORCAS DOUGLAS KIBAI', 'Destelia  Malela', 'JOAN MOSHI', 'JOSHUA PALANGYO', 'GIFT STANFOD', 'SHARON NASARY']
2025-03-07: 4 students absent: ['MELVIN NDEKIRWA KITOMARI', 'DOREEN JOSEPHAT KITOMARI', 'MARTIN KIMUTTO', 'GEORGE AKYOO']
2025-03-14: 7 students absent: ['JOHNSON THOMAS AKYOO', 'MELVIN NDEKIRWA KITOMARI', 'GODLOVE KAANANKIRA KITOMARI', 'ANTELGOD PALANGYO', 'MARTIN KIMUTTO', 'SOPHIA AKYOO', 'DEBORA KYUNGAI']
2025-03-21: 1 students absent: ['DOREEN URIO']


In [4]:
# Define the folder path containing CSV files
folder_path = DIR_LEVELS

# Define level order for plotting
level_order = ['Introductory'] + [f"Level {str(i)}" for i in range(1, 26)]
level_mapping = {level: i for i, level in enumerate(level_order)}

# Function to extract stream name from filename
def extract_stream(filename):
    list_of_streams = ["3A", "3B", "A", "B"] # this order to avoid 3A being detected as A
    for stream in list_of_streams:
        if stream in filename:
            return stream
    return None

# Function to clean and convert Sync date
def clean_sync_date(date_value):
    """Convert valid date strings to datetime, ignore relative timestamps."""
    try:
        return pd.to_datetime(date_value, errors="coerce")  # Convert or return NaT
    except Exception:
        return np.nan

# Dictionary to store dataframes per stream
stream_data = {"A": [], "B": [], "3A": [], "3B": []}

kids_not_in_program = []
# Read all CSV files and segment by stream
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        stream = extract_stream(file)
        if stream:
            df = pd.read_csv(os.path.join(folder_path, file))
            # check if the student is in the list of students
            #kids_not_in_program += [kid for kid in df["Name"].unique() if kid not in all_kids_2025["STUDENTS NAME"].values and
            #                         kid.capitalize() not in all_kids_2025["STUDENTS NAME"].values and
            #                         kid not in kids_not_in_program]

            # New format of file from Sept 25: 1st row is not the columns, need to read from 2nd row
            if "Name" not in df.columns:
                df = pd.read_csv(os.path.join(folder_path, file), header=1)
            # remove kids that left the program
            df_cleaned = df[~df["Name"].isin(oop_kids["Name"].values)].copy()
            if len(df) != len(df_cleaned):
                print(f"Removed {df.shape[0] - df_cleaned.shape[0]} kids who left the program from {file}")

            # Apply cleaning function to Sync date
            # print name of csv file
            print(f"Processing {file}")
            df_cleaned["Sync date"] = df_cleaned["Sync date"].apply(clean_sync_date)
            # Drop rows where Sync date couldn't be parsed
            #df = df.dropna(subset=["Sync date"])
            # Ensure Level is treated as a categorical variable
            df_cleaned["Level"] = pd.Categorical(df_cleaned["Level"], categories=level_order, ordered=True)
            # change type of Level to category
            #df["Level"] = df["Level"].astype("category")
            stream_data[stream].append(df_cleaned)

stream_data_grade = {
    "Grade 3": stream_data["3A"] + stream_data["3B"],
    "Grade 4": stream_data["A"] + stream_data["B"]
}

Removed 11 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250204.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250204.csv
Removed 11 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250207.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250207.csv
Removed 9 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250214.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250214.csv
Removed 8 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250222.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250222.csv
Removed 8 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250301.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250301.csv
Removed 8 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250308.csv
Processing E3Empower_class_English_E-Shuleni 3A_20250308.csv
Removed 8 kids who left the program from E3Empower_class_English_E-Shuleni 3A_20250316

In [5]:
kids_from_csv = pd.DataFrame()
for stream, data in stream_data.items():
    for df in data:
        tmp = df[['Name']].copy()
        tmp.loc[:, 'Stream'] = stream
        kids_from_csv = pd.concat([kids_from_csv, tmp], ignore_index=True)
print(kids_from_csv.shape)
#print(f"Number of students in the program from CSV files: {len(kids_from_csv['Name'].unique())}")
# keep only unique names
kids_from_csv = kids_from_csv.drop_duplicates(subset=['Name']).reset_index(drop=True)
print(kids_from_csv.shape)
print(f"Number of unique students in the program from CSV files: {len(kids_from_csv)}")
print(kids_from_csv['Stream'].value_counts())
kids_from_csv.head()

(4510, 2)
(166, 2)
Number of unique students in the program from CSV files: 166
Stream
3A    55
B     43
A     38
3B    30
Name: count, dtype: int64


,Name,Stream
0,BRAYAN JOSHUA KIMUTHO,A
1,CATHERINE GODLISTEN,A
2,EDGAR ARON MANANG,A
3,GREYSON WILBERT KITOMARI,A
4,HERBERT SENYAEL KITOMARI,A


In [6]:
all_kids_2025 = {}

def plot_learning_progress(stream_data_grade, grade_level, level_order, kids_not_in_program):

    #for grade, dfs in stream_data_grade.items():
    #if not dfs:
    #    continue  # Skip if no files for this stream
    dfs = stream_data_grade[grade_level]
    if not dfs:
        print(f"No data for {grade_level}")
        return None, []

    # Concatenate all dataframes for this stream
    combined_df = pd.concat(dfs, ignore_index=True).reset_index(drop=True)
    # Sort by Name and Sync date for proper line plotting
    combined_df = combined_df.sort_values(by=["Name", "Sync date"])
    # Convert Levels to numerical values for plotting
    combined_df["Level_num"] = combined_df["Level"].cat.codes
    # remove kids not in program
    combined_df = combined_df[~combined_df["Name"].isin(kids_not_in_program)]
    # remove NaN values = rows with -1 in column "Level_num"
    combined_df = combined_df[combined_df["Level_num"] != -1].reset_index(drop=True)

    # Temporary: remove HILARY JOSEPHAT KITOMARI from the dataset
    combined_df = combined_df[combined_df["Name"] != "HILARY JOSEPHAT KITOMARI"]

    # print number of unique Name
    #print(f"Number of unique students in {grade}: {combined_df['Name'].nunique()}")
    # save kids as ground truth for the dashboard (plots perf and number of lessons)
    #all_kids_2025[grade] = combined_df["Name"].unique().tolist()
    list_kids_names = combined_df["Name"].unique().tolist()
    # Create interactive line plot
    fig = px.line(
        combined_df,
        x="Sync date",
        y="Level_num",
        color="Name",  # Differentiating by student name
        markers=True,
        title=f"Learning Progress for {grade_level} - Level",
        labels={"Sync date": "Date", "Level_num": "Level"},
    )
    # Customize y-axis to display original level names
    fig.update_yaxes(
        tickmode="array",
        tickvals=list(range(len(level_order))),
        ticktext=level_order
    )
    # Improve layout
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Level",
        xaxis=dict(tickangle=-45),
        template="plotly_white"
    )
    # save in html format
    #fig.write_html(DIR_PLOTS_DASHBOARD + f"tracking_levels_{grade}.html")

    # add a grey zone between 7th and 20th of april for holidays
    fig.add_shape(
        type="rect",
        x0="2025-04-07",
        y0=-0.5,
        x1="2025-04-20",
        y1=max(combined_df["Level_num"]),
        fillcolor="grey",
        opacity=0.2,
        line_width=0,
    )

    # add a grey zone between 2nd of June to 6th of July for holidays
    fig.add_shape(
        type="rect",
        x0="2025-06-02",
        y0=-0.5,
        x1="2025-07-06",
        y1=max(combined_df["Level_num"]),
        fillcolor="grey",
        opacity=0.2,
        line_width=0,
    )

    return fig, list_kids_names, combined_df


# Plot learning progress for each grade and save the figure
fig_level_grade3, list_kids_names_grade3, levels_aggregated_grade3 = plot_learning_progress(stream_data_grade, "Grade 3", level_order, kids_not_in_program)
fig_level_grade4, list_kids_names_grade4, levels_aggregated_grade4 = plot_learning_progress(stream_data_grade, "Grade 4", level_order, kids_not_in_program)
fig_level_grade3.show()
fig_level_grade4.show()

In [7]:
# save figs
fig_level_grade3.write_html(DIR_PLOTS_DASHBOARD + "tracking_levels_Grade 3.html")
fig_level_grade4.write_html(DIR_PLOTS_DASHBOARD + "tracking_levels_Grade 4.html")

**Add Level/week metrics to track**

In [9]:
start_date, end_date = levels_aggregated_grade3["Sync date"].min(), levels_aggregated_grade3["Sync date"].max()
#print(f"Start date: {start_date}, End date: {end_date}")

# add a table with mean number of levels passed per week for each grade,
# for each kid, look at start and end date and start and end level, calculate the number of levels passed per week
df_levels_per_week = pd.DataFrame()
# store all kids from both grades from the levels_aggregated dataframes, add column for grade
df_levels_per_week = pd.concat([levels_aggregated_grade3.assign(Grade="Grade 3"), levels_aggregated_grade4.assign(Grade="Grade 4")], ignore_index=True)
print(df_levels_per_week.Grade.value_counts())
df_levels_per_week.head()

Grade
Grade 3    2237
Grade 4    2129
Name: count, dtype: int64


,Name,Sync date,Device Sync Needed,Landing level,Level,Unit,Unit objective,Level_num,Grade
0,AIVIN URIO,2025-01-21,Yes,-,Introductory,Unit 1,Becoming comfortable with English,0,Grade 3
1,AIVIN URIO,2025-02-06,No,Level 3,Level 3,Unit 1,"Sounds: a, t, i, p and Feelings",3,Grade 3
2,AIVIN URIO,2025-02-14,No,Level 3,Level 3,Unit 3,"Sounds: d, m, c, u, a, n and Things that Move",3,Grade 3
3,AIVIN URIO,2025-02-21,No,Level 3,Level 3,Review 3,Sounds: d / m / c / u / a / n and Things that ...,3,Grade 3
4,AIVIN URIO,2025-02-28,No,Level 3,Level 4,Unit 1,Sounds: k / h / o / d / t and Actions,4,Grade 3


In [10]:
df_levels_per_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4366 entries, 0 to 4365
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                4366 non-null   object        
 1   Sync date           4132 non-null   datetime64[ns]
 2   Device Sync Needed  4366 non-null   object        
 3   Landing level       4366 non-null   object        
 4   Level               4366 non-null   category      
 5   Unit                4366 non-null   object        
 6   Unit objective      4366 non-null   object        
 7   Level_num           4366 non-null   int8          
 8   Grade               4366 non-null   object        
dtypes: category(1), datetime64[ns](1), int8(1), object(6)
memory usage: 248.7+ KB


In [11]:
def extract_bottom_levels(levels_aggregated, grade_level, time, N_kids=10):
    """
    Extract the names of kids that are in the lowest levels for a given time.
    """
    # filter by time
    levels_aggregated = levels_aggregated[levels_aggregated["Sync date"] == time].reset_index(drop=True)
    # get the bottom N_kids
    bottom_levels = levels_aggregated.nsmallest(N_kids, "Level_num")
    # get the names of the kids
    bottom_kids = bottom_levels["Name"].tolist()
    return bottom_kids

# get the last date in the dataframe
last_date_grade3 = levels_aggregated_grade3["Sync date"].max()
last_date_grade4 = levels_aggregated_grade4["Sync date"].max()

bottom_kids_grade3 = extract_bottom_levels(levels_aggregated_grade3, "Grade 3", last_date_grade3, N_kids=10)
bottom_kids_grade4 = extract_bottom_levels(levels_aggregated_grade4, "Grade 4", last_date_grade4, N_kids=10)
print(f"Bottom kids in Grade 3 on {last_date_grade3}: {bottom_kids_grade3}")
print(f"Bottom kids in Grade 4 on {last_date_grade4}: {bottom_kids_grade4}")

Bottom kids in Grade 3 on 2025-11-13 00:00:00: ['Arafat Jonas', 'Furaha Kitomari', 'Maximillian  Revocatus', 'Shedrack Kitomari', 'DANIEL AKYOO', 'Alice Muhando', 'EBENEZER KITOMARI', 'GODSON KITUNDU', 'GRACIOUS NYITI', 'HILLARY KIMUTO']
Bottom kids in Grade 4 on 2025-11-13 00:00:00: ['Beautiful Godman Akyoo', 'HERBERT SENYAEL KITOMARI', 'BRAYAN JOSHUA KIMUTHO', 'DOREEN JOSEPHAT KITOMARI', 'GODLOVE KAANANKIRA KITOMARI', 'VIOLA MARKO  AKYOO', 'ARSON BARAKA AKYOO', 'GEORGE ELIFAS PALLANGYO', 'PENDO PETRO KITOMARI', 'ZULEKHA MOHAMED MURO']


## Track Performance

In [12]:
# Define the folder path containing CSV files
DIR_PERF = "./../Pilot 2025/Data Enuma LMS/track performance/"
folder_path = DIR_PERF

# create dict with all kids from csv files for both grades
all_kids_2025 = {}
all_kids_2025["Grade 3"] = list_kids_names_grade3
all_kids_2025["Grade 4"] = list_kids_names_grade4
print(f"Total number of unique students in both grades: {len(set(all_kids_2025['Grade 3'] + all_kids_2025['Grade 4']))}")
    
# create function to clean the date
def clean_date(date_value):
    month_mapping = {"jan": "01", "feb": "02", "mar": "03", "apr": "04", "may": "05", "jun": "06", "jul": "07", "aug": "08", "sep": "09", "oct": "10", "nov": "11", "dec": "12"}
    try:
        month, day = date_value.split(" ")
        month = month_mapping[month.lower()]
        return f"2025-{month}-{day}"
    except Exception:
        return pd.NaT


month_mapping = {"jan": "01", "feb": "02", "mar": "03", "apr": "04", "may": "05", "jun": "06", "jul": "07", "aug": "08", "sep": "09", "oct": "10", "nov": "11", "dec": "12"}

kids_missing_performance = pd.DataFrame()
df_perf = pd.DataFrame()
kids_from_pdf = []

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        # extract date from filename
        month = file.split("_")[2]
        day = file.split("_")[-1].replace(".xlsx", "")

        month_num = month_mapping[month.lower()]
        date = f"2025-{month_num}-{day}"
        
        df = pd.read_excel(os.path.join(folder_path, file))
        # remove HILARY JOSEPHAT KITOMARI from dataframe if present
        #if "HILARY JOSEPHAT KITOMARI" in df["Name"].values:    
        #    df = df[df["Name"] != "HILARY JOSEPHAT KITOMARI"].reset_index(drop=True)
        #    print(f"Removed HILARY JOSEPHAT KITOMARI from {file}")

        # replace NaN values in columns Overall Percentage Score
        df['Overall Percentage Score'] = df['Overall Percentage Score'].fillna(0)
        
        # clean rows with KITOMARI
        df['Name'] = df['Name'].str.replace("KITOMA RI", "KITOMARI")
        df['Name'] = df['Name'].str.replace("JOSH UA", "JOSHUA")
        df['Name'] = df['Name'].str.replace("KITOMAR I", "KITOMARI")
        df['Name'] = df['Name'].str.replace("OME GA", "OMEGA")
        df['Name'] = df['Name'].str.replace("MOSH I	", "MOSHI")
        df['Name'] = df['Name'].str.replace("Benedic t", "Benedict")
        # remove 2 spaces with 1 space
        #df['Name'] = df['Name'].str.replace("  ", " ")
        #df['Name'] = df['Name'].str.replace("Destelia\xa0", "Destelia")

        # remove kids that left the program
        df_cleaned = df[~df["Name"].isin(oop_kids["Name"].values)].copy()
        if df.Name.nunique() != df_cleaned.Name.nunique():
            print(f"Removed {df.Name.nunique() - df_cleaned.Name.nunique()} kids who left the program from {file}")

        # collect names of all kids from pdf
        kids_from_pdf += df_cleaned["Name"].unique().tolist()
        # remove duplicates from list of kids from pdf
        kids_from_pdf = list(set(kids_from_pdf))

        # loop through the list of students in program and save date and performance
        #for student in all_kids_2025["STUDENTS NAME"].values:
        # move HILARY JOSEPHAT KITOMARI from Grade 3 to Grade 4 for the performance report
       
        for student in all_kids_2025["Grade 3"] + all_kids_2025["Grade 4"]:
        #for student in df["Name"].values:
            if student in df_cleaned["Name"].values:
                score = df_cleaned[df_cleaned["Name"] == student]["Overall Percentage Score"].values[0]
                lesson_completed = df_cleaned[df_cleaned["Name"] == student]["Lesson Completed"].values[0]
                stream = df_cleaned[df_cleaned["Name"] == student]["Class"].values[0]
                priority = df_cleaned[df_cleaned["Name"] == student]["Priority Perf"].values[0]

                # add absent kids from absenteeism dict
                if student in absenteeism_dict.get(date, []):
                    priority = "Absent" # absent student
                #if score != 0:
                df_perf = pd.concat([df_perf, pd.DataFrame({"Name": student, "Stream": stream, "Date": date, "Performance": score, "Lesson Completed": lesson_completed, "Priority Perf": priority}, index=[0])], ignore_index=True)
                #else:
                    # set score to 0 (missing score equivalent to 0)
                    #df_perf = pd.concat([df_perf, pd.DataFrame({"Name": student, "Stream": stream, "Date": date, "Performance": 0, "Lesson Completed": 0}, index=[0])], ignore_index=True)
            else:
                # Student not in the program at this date
                kids_missing_performance = pd.concat([kids_missing_performance, pd.DataFrame({"Name": student, "Date": date}, index=[0])], ignore_index=True)

# pd.to_datetime
df_perf["Date"] = pd.to_datetime(df_perf["Date"])
kids_missing_performance["Date"] = pd.to_datetime(kids_missing_performance["Date"])

# combine streams by Grades
df_perf["Grade"] = df_perf["Stream"].apply(lambda x: "Grade 3" if x in ["3A", "3B"] else "Grade 4")

print(df_perf.shape, kids_missing_performance.shape)
display(df_perf.head(5), kids_missing_performance.head(2))
print(f"Kids ({kids_missing_performance['Name'].nunique()}) that could not be matched between csv and pdf progress reports: \n{kids_missing_performance['Name'].unique().tolist()}")
print(f"\nDifferent dates: {df_perf['Date'].unique()}")

Total number of unique students in both grades: 164
Removed 8 kids who left the program from progress_reports_apr_all_streams_4.xlsx
Removed 8 kids who left the program from progress_reports_apr_all_streams_4_25.xlsx
Removed 2 kids who left the program from progress_reports_aug_all_streams_15_22.xlsx
Removed 2 kids who left the program from progress_reports_aug_all_streams_8.xlsx
Removed 2 kids who left the program from progress_reports_aug_all_streams_8_15.xlsx
Removed 14 kids who left the program from progress_reports_feb_all_streams_14_21.xlsx
Removed 9 kids who left the program from progress_reports_feb_all_streams_21_28.xlsx
Removed 17 kids who left the program from progress_reports_feb_all_streams_7.xlsx
Removed 17 kids who left the program from progress_reports_feb_all_streams_7_14.xlsx
Removed 16 kids who left the program from progress_reports_jan_all_streams_31.xlsx
Removed 4 kids who left the program from progress_reports_jul_all_streams_11.xlsx
Removed 2 kids who left the pr

,Name,Stream,Date,Performance,Lesson Completed,Priority Perf,Grade
0,AIVIN URIO,3A,2025-04-04,100.0,6,3,Grade 3
1,ALICE AKYOO,3A,2025-04-04,0.0,0,0,Grade 3
2,ANGEL KITOMARI,3A,2025-04-04,100.0,6,3,Grade 3
3,ANJELA KITOMARI,3B,2025-04-04,0.0,4,1,Grade 3
4,ANTELGOD PALANGYO,3A,2025-04-04,100.0,10,3,Grade 3


,Name,Date
0,Arafat Jonas,2025-04-04
1,JOAN MOSHI,2025-04-04


Kids (22) that could not be matched between csv and pdf progress reports: 
['Arafat Jonas', 'JOAN MOSHI', 'Maximillian  Revocatus', 'Shedrack Kitomari', 'Yunis Judika Akyoo', 'Innocent Stefano Mwanja', 'SALOME MICHAEL IPOLO', 'VIOLA MARKO  AKYOO', 'VOCADEAR MELCHIORY MAMBO', 'GODLOVE KAANANKIRA KITOMARI', 'Bryton Nelson', 'Destelia\xa0 Malela', 'Praygod Benedict', 'Sharon Jastin', 'Alice Muhando', 'Angel Kaaya', 'Arnold Mlay', 'Christian Mtei', 'Daniel Zitto', 'Furaha Kitomari', 'Jackson Shila', 'Beautiful Godman Akyoo']

Different dates: <DatetimeArray>
['2025-04-04 00:00:00', '2025-04-25 00:00:00', '2025-08-22 00:00:00',
 '2025-08-08 00:00:00', '2025-08-15 00:00:00', '2025-02-21 00:00:00',
 '2025-02-28 00:00:00', '2025-02-07 00:00:00', '2025-02-14 00:00:00',
 '2025-01-31 00:00:00', '2025-07-11 00:00:00', '2025-07-18 00:00:00',
 '2025-07-25 00:00:00', '2025-07-31 00:00:00', '2025-03-21 00:00:00',
 '2025-03-28 00:00:00', '2025-03-07 00:00:00', '2025-03-14 00:00:00',
 '2025-05-23 00:00:

### Printing in dashboard the last sync date of students with priority 0

In [13]:
# for each student, get the last date and the last performance
df_last_priority = df_perf.groupby(["Name", "Grade"]).agg({"Date": "max", "Performance": "last", "Lesson Completed": "last", "Priority Perf": "last"}).reset_index()
df_last_priority_0 = df_last_priority[df_last_priority["Performance"] == 0].copy()
kids_priority_0_last_date = df_last_priority_0["Name"].unique().tolist()

### Fixing names of students not included in dashboard

Reason: some kids' names were badly extracted from pdf extraction

In [15]:
# for each stream, plot the number of words in column Name, split space and count words
# create a new column with number of words in Name
#df_perf["Name_words"] = df_perf["Name"].str.split().apply(len)
# print 

#sns.countplot(data=df_perf, x="Name_words", hue="Stream")
#plt.show()

In [16]:
#print(f"Number of unique students in the program from PDF: {len(kids_from_pdf)}")
#print(f"Number of unique students in the program from CSV: {len(kids_from_csv)}")
#
#df_names_pdf_csv = pd.DataFrame()
#df_names_pdf_csv["Name pdf"] = list(set(kids_from_pdf))
#for student in df_names_pdf_csv["Name pdf"]:
#    if student in kids_from_csv["Name"].values:
#        df_names_pdf_csv.loc[df_names_pdf_csv["Name pdf"] == student, "Name csv"] = student
#        df_names_pdf_csv.loc[df_names_pdf_csv["Name pdf"] == student, "Stream csv"] = kids_from_csv[kids_from_csv["Name"] == student]["Stream"].values[0]
#    else:
#        df_names_pdf_csv.loc[df_names_pdf_csv["Name pdf"] == student, "Name csv"] = np.nan
#        df_names_pdf_csv.loc[df_names_pdf_csv["Name pdf"] == student, "Stream csv"] = np.nan
#print(df_names_pdf_csv['Stream csv'].value_counts())
#
## non matched students between pdf and csv
#bad_matches = df_names_pdf_csv[df_names_pdf_csv["Name csv"].isna()]
#print(f"Number of students not matched between pdf and csv: {bad_matches.shape[0]}")
#bad_matches.head(len(bad_matches))

In [15]:
#for name in bad_matches["Name pdf"].values:
#    print(f"Name: {name}, (type: {type(name)})")

In [14]:
#from thefuzz import fuzz
#
## Function to find the best match for a name in a list
#def find_best_match(name, name_list):
#    best_match = None
#    for candidate in name_list:
#        ratio = fuzz.ratio(name.lower(), candidate.lower())
#        if ratio > 80:
#            best_match = candidate
#    return best_match
#
## Find best matches for non-matched students
#bad_matches['Best Match'] = bad_matches['Name pdf'].apply(lambda x: find_best_match(x, df_names_pdf_csv['Name csv'].dropna().tolist()))
#bad_matches.head(len(bad_matches))

In [17]:
def plot_perf_timeline(df_perf, grade, yaxis):
    fig = px.line(
            # take df_perf grouped by name and data sorted
            df_perf.sort_values(by=["Name", "Date"]),
            x="Date",
            y=yaxis,
            color="Name",  # Differentiating by student name
            markers=True,
            title=f"Learning Progress for {grade} - Overall Percentage Score" if yaxis == "Performance" else f"Learning Progress for {grade} - Number of Lessons Completed",
        )
    # Customize y-axis to display original level names
    if yaxis == "Performance":
        fig.update_yaxes(
            tickmode="array",
            tickvals=list(range(0, 101, 20)),
            ticktext=list(range(0, 101, 20)),
        )
    else:
        fig.update_yaxes(
                tickmode="array",
            )
    # Improve layout
    fig.update_layout(
            xaxis_title="Date",
            yaxis_title= "Overall Percentage Score" if yaxis == "Performance" else "Number of Lessons Completed",
            xaxis=dict(tickangle=-45),
            template="plotly_white"
        )
    
    # add a grey zone between 7th and 20th of april for holidays
    fig.add_shape(
        type="rect",
        x0="2025-04-07",
        y0=-0.5,
        x1="2025-04-20",
        y1=100 if yaxis == "Performance" else df_perf[yaxis].max() + 0.5,
        fillcolor="grey",
        opacity=0.2,
        line_width=0,
    )
    # add a grey zone between 2nd of June to 6th of July for holidays
    fig.add_shape(
        type="rect",
        x0="2025-06-02",
        y0=-0.5,
        x1="2025-07-06",
        y1=100 if yaxis == "Performance" else df_perf[yaxis].max() + 0.5,
        fillcolor="grey",
        opacity=0.2,
        line_width=0,
    )

    return fig

def plot_perf_timeline_bars(df_perf, grade):
    # Define the custom color mapping
    dict_colors = {
        "0": "purple",
        "1": "red",
        "2": "orange",
        "3": "green",
        "Absent": "blue",
    }
    
    # Group by date, stream, and priority perf to get the frequency
    grouped = (
        df_perf
        .groupby(["Date", "Stream", "Priority Perf"])
        .size()
        .reset_index(name="Count")
    )
    # Convert Priority Perf to string, so it matches the keys in dict_colors
    grouped["Priority Perf"] = grouped["Priority Perf"].astype(str)

    # --- This section replaces the 'todo' ---
    # Convert Date to a string ('MM-DD') for a categorical x-axis.
    # This is the key step to remove any gaps in the date sequence.
    #grouped['Date_str'] = grouped['Date'].dt.strftime('%m-%d')
    #
    ## To keep chronological order, we create a sorted list of the unique date strings.
    ## We sort based on the original datetime 'Date' column.
    #unique_dates_sorted = sorted(grouped['Date'].unique())
    #unique_dates_str_sorted = [d.strftime('%m-%d') for d in unique_dates_sorted]
    # --- End of section ---
    
    # Create a bar chart with custom colors, and separate subplots by Stream
    fig = px.line(
        grouped,
        x="Date",
        y="Count",
        color="Priority Perf",
        facet_col="Stream",          
        #barmode="group",
        color_discrete_map=dict_colors,
        title=f"Learning Progress for {grade} - Overall Percentage Score",
        # Explicitly set the category order to ensure dates are not sorted alphabetically
        #category_orders={"Date_str": unique_dates_str_sorted}
    )
    
    # Rotate and relabel the x‐axis ticks
    fig.update_xaxes(
        #type='category',
        tickangle=-45,
        tickvals=grouped['Date'].unique(),
        #tickvals=grouped['Date'].astype(str).unique(),
        ticktext=grouped['Date'].dt.strftime('%m-%d').unique()
    )
    
    # General layout updates
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Count",
        template="plotly_white"
    )

    # rename label 4 abscent for priority perf
    #fig.for_each_annotation(lambda a: a.update(text=a.text.replace("4", "Absent")))
    return fig

In [18]:
fig_grade3_perf = plot_perf_timeline_bars(df_perf[df_perf["Grade"] == "Grade 3"], "Grade 3")
fig_grade4_perf = plot_perf_timeline_bars(df_perf[df_perf["Grade"] == "Grade 4"], "Grade 4")
fig_grade3_perf.show()
fig_grade4_perf.show()

In [19]:
# save in html format
fig_grade3_perf.write_html(DIR_PLOTS_DASHBOARD + f"tracking_perf_Grade 3.html")
fig_grade4_perf.write_html(DIR_PLOTS_DASHBOARD + f"tracking_perf_Grade 4.html")

## Dashboard on Number of lessons completed

In [20]:
fig_grade3_lesson = plot_perf_timeline(df_perf[df_perf["Grade"] == "Grade 3"], "Grade 3", yaxis="Lesson Completed")
fig_grade4_lesson = plot_perf_timeline(df_perf[df_perf["Grade"] == "Grade 4"], "Grade 4", yaxis="Lesson Completed")

fig_grade3_lesson.show()
fig_grade4_lesson.show()

In [21]:
# save in html format
fig_grade3_lesson.write_html(DIR_PLOTS_DASHBOARD + f"tracking_lessons_Grade 3.html")
fig_grade4_lesson.write_html(DIR_PLOTS_DASHBOARD + f"tracking_lessons_Grade 4.html")

## Add page for Priority 0 students

- add last sync date
- add plots for level evolution
- add plot number of lesson
- add history of priority groups

In [22]:
# identify kids with 0 in Priority Perf in the last date
kids_p0_group = []

for kid in df_perf["Name"].unique():
    df_perf_kid = df_perf[df_perf["Name"] == kid]
    # get the last date for the kid
    last_date = df_perf_kid["Date"].max()
    # get the Priority Perf for the last date
    priority_perf = df_perf_kid[df_perf_kid["Date"] == last_date]["Priority Perf"].values[0] if not df_perf_kid[df_perf_kid["Date"] == last_date].empty else None
    # if Priority Perf is 0, add the kid to the group
    if priority_perf == 0:
        kids_p0_group.append(kid)

print(f"Number of kids with 0 in Priority Perf in the last date: {len(kids_p0_group)}")

# save kids with 0 in Priority Perf in the last date
df_perf_p0 = df_perf[df_perf["Name"].isin(kids_p0_group)].copy().reset_index(drop=True)

fig_grade3_lesson_p0 = plot_perf_timeline(df_perf_p0[df_perf_p0["Grade"] == "Grade 3"], "Grade 3", yaxis="Lesson Completed")
fig_grade4_lesson_p0 = plot_perf_timeline(df_perf_p0[df_perf_p0["Grade"] == "Grade 4"], "Grade 4", yaxis="Lesson Completed")

fig_grade3_lesson_p0.show()
fig_grade4_lesson_p0.show()

Number of kids with 0 in Priority Perf in the last date: 81


In [23]:
# save in html format
fig_grade3_lesson_p0.write_html(DIR_PLOTS_DASHBOARD + f"tracking_lessons_Grade 3_p0.html")
fig_grade4_lesson_p0.write_html(DIR_PLOTS_DASHBOARD + f"tracking_lessons_Grade 4_p0.html")

In [24]:
# create a heatmap for each kids at bottom, for each grade. Each row is one kid, one column is Priority Perf from all the dates
def create_heatmap(df_perf, bottom_kids, grade):
    # Filter the dataframe for bottom kids
    df_bottom = df_perf[df_perf["Name"].isin(bottom_kids) & (df_perf["Grade"] == grade)]
    
    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = df_bottom.pivot_table(
        index="Name",
        columns="Date",
        values="Priority Perf",
        aggfunc='first'  # Use first value in case of duplicates
    )
    custom_palette = {
        "0": "purple",
        "1": "red",
        "2": "orange",
        "3": "green",
        "4": "blue"
    }
    # Create a heatmap using Plotly
    fig = px.imshow(
        heatmap_data,
        color_continuous_scale=[custom_palette["0"], custom_palette["1"], custom_palette["2"], custom_palette["3"], custom_palette["4"]],
        title=f"Heatmap of Priority 0 Kids in {grade}",
        labels=dict(x="", y="Student", color="Priority Perf"),
    )
    # reduce the size of the figure for better readability
    fig.update_layout(
        height=300,
        )

    # Update layout for better readability
    fig.update_xaxes(tickangle=-45)
    fig.update_yaxes(title_text="Students",
                     tickfont=dict(size=8))
    return fig

In [25]:
def create_heatmap(df_perf, bottom_kids, grade):
    """
    Generates a heatmap with discrete, equal-sized cells for each performance category.
    """
    # Filter the dataframe for bottom kids
    df_bottom = df_perf[df_perf["Name"].isin(bottom_kids) & (df_perf["Grade"] == grade)].copy()
    
    # Ensure 'Date' is in datetime format for correct sorting
    df_bottom['Date'] = pd.to_datetime(df_bottom['Date'])

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = df_bottom.pivot_table(
        index="Name",
        columns="Date",
        values="Priority Perf",
        aggfunc='first'  # Use first value in case of duplicates
    )

    # Sort columns chronologically
    heatmap_data = heatmap_data.reindex(sorted(heatmap_data.columns), axis=1)

    # Define the discrete colors for each performance level.
    custom_palette = {
        0: "purple",
        1: "red",
        2: "orange",
        3: "green",
        4: "blue"  # This likely corresponds to 'Absent'
    }

    # Create a discrete colorscale to prevent a gradient
    zmin = 0
    zmax = 4
    colorscale = [
        [0.0, custom_palette[0]], [0.125, custom_palette[0]],
        [0.125, custom_palette[1]], [0.375, custom_palette[1]],
        [0.375, custom_palette[2]], [0.625, custom_palette[2]],
        [0.625, custom_palette[3]], [0.875, custom_palette[3]],
        [0.875, custom_palette[4]], [1.0, custom_palette[4]]
    ]

    # --- FIX: Use go.Heatmap for equal-sized cells ---
    # Convert datetime columns to strings to create a categorical axis
    x_axis_labels = heatmap_data.columns.strftime('%Y-%m-%d')

    # Create the heatmap using the more fundamental go.Heatmap object.
    # This gives us direct control and ensures each column is treated as a
    # separate category with equal width, creating uniform boxes.
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=x_axis_labels,
        y=heatmap_data.index,
        colorscale=colorscale,
        zmin=zmin,
        zmax=zmax,
        # Add a small gap between cells for better visual separation
        xgap=1,
        ygap=1
    ))

    print(df_bottom["Name"].nunique())
    # Update layout to match the previous style
    fig.update_layout(
        title=f"Heatmap of Priority 0 Kids in {grade}",
        height=80*df_bottom["Name"].nunique(),  # Adjust height based on number of students
        coloraxis_colorbar=dict(
            title="Priority Perf",
            tickvals=[0, 1, 2, 3, 4],
            ticktext=["0", "1", "2", "3", "4 (Absent)"],
        )
    )
    # --- End of fix ---

    # Update layout for better readability
    fig.update_xaxes(title_text="Date", tickangle=-45)
    fig.update_yaxes(title_text="Students", tickfont=dict(size=8))
    
    return fig

In [26]:
# represent in the heatmap, one row per kid, one column per date, color by Priority Perf
fig_heatmap_grade3_p0 = create_heatmap(df_perf_p0, kids_p0_group, "Grade 3")
fig_heatmap_grade4_p0 = create_heatmap(df_perf_p0, kids_p0_group, "Grade 4")
fig_heatmap_grade3_p0.show()
fig_heatmap_grade4_p0.show()

5
76


In [27]:
# save fig
fig_heatmap_grade3_p0.write_html(DIR_PLOTS_DASHBOARD + f"tracking_heatmap_Grade 3_p0.html")
fig_heatmap_grade4_p0.write_html(DIR_PLOTS_DASHBOARD + f"tracking_heatmap_Grade 4_p0.html")